In [1]:
cd ../src

C:\Users\jeong\Desktop\python\action-recognition-practice\src


In [2]:
from glob import glob
from os import path
import numpy as np
import matplotlib.pyplot as plt
import torchvision.datasets as datasets
from torchvision.datasets.utils import list_dir
from tqdm import tqdm
from processing.video import *
import ffmpeg
import random
from IPython.display import display, HTML
from matplotlib import animation

In [3]:
raw_path = "../data/hmdb/raw"
processed_path = "../data/hmdb/processed"

In [4]:
videos = glob(os.path.join(raw_path, "**/*.avi"))
videos = list(map(lambda p: path.abspath(p), videos))

In [5]:
path.basename(path.dirname(videos[0]))

'climb_stairs'

In [46]:
clips = []
targets = []
for v in tqdm(videos):
    clip = create_clip(v, 8, resolution="112x112")
    target = path.basename(path.dirname(v))
    clips.append(clip)
    targets.append(target)

100%|██████████| 3919/3919 [08:36<00:00,  7.59it/s]


In [61]:
np.savez_compressed(path.join(processed_path, "subset"), clips=np.array(clips), targets=np.array(targets))

In [4]:
loaded = np.load(path.join(processed_path, "subset.npz"))
clips_loaded = loaded['clips']
targets_loaded = loaded['targets']
loaded['clips'].shape, loaded['targets'].shape

((3919, 3, 8, 112, 112), (3919,))

In [5]:
def play_as_video(clip, target):
    video = np.einsum("cthw->thwc", clip)
    fig = plt.figure()
    im = plt.imshow(video[0,:,:,:])
    plt.title(target)
    plt.close() # this is required to not display the generated image

    def init():
        im.set_data(video[0,:,:,:])

    def animate(i):
        im.set_data(video[i,:,:,:])
        return im

    anim = animation.FuncAnimation(fig, animate, frames=video.shape[0],
                                interval=1000/3)
    return HTML(anim.to_html5_video())

In [7]:
for _ in range(30):
    idx = random.randrange(len(clips_loaded))
    display(play_as_video(clips_loaded[idx], targets_loaded[idx]))